In [ ]:
from gym import envs
all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print(sorted(env_ids))

In [ ]:
import numpy as np
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete), \
            "Should only be used to wrap Discrete envs."
        self.n = self.observation_space.n
        self.observation_space = gym.spaces.Box(0, 1, (self.n,))
    
    def observation(self, obs):
        new_obs = np.zeros(self.n)
        new_obs[obs] = 1
        return new_obs

In [ ]:


import gym
env = ObservationWrapper(gym.make('Taxi-v3'))
#env = gym.make('Taxi-v3')
print(env.action_space)
print(env.observation_space)



env = gym.make('CartPole-v0')
print(env.action_space)
print(env.observation_space)





In [8]:
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
env=jss_lite(instance_path='resources/jsp_instances/standard/abz5.txt')

token=False
state=env.reset()
for i in range(999999):
    legal_action=env.get_legal_actions(state)
    action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
    if action > env.action_space.n/2:
        token=True
        print("before:")
        env.render(start_count=1,x_bar="Job",y_bar="Machine")
        state, reward, done, info=env.step(action)


    else:
        state, reward, done, info=env.step(action)
        if token:
            token=False
            print("after:")
            env.render(start_count=1,x_bar="Job",y_bar="Machine")
    #print(action)
    #tmp=(env.processed_and_max_time_job_matrix)
    #print(env.count_finished_tasks_machine_matrix)
    #print(env.production_list)
    #env.render()
    if done==True:
        print(f"finished! no legal actions possible after {i+1} steps")
        break
env.render(start_count=1,x_bar="Job",y_bar="Machine")



before:


after:


before:


after:


before:


...

before:


after:


before:


after:


finished! no legal actions possible after 122 steps


In [4]:
pro=env.production_list

In [3]:
env.render()
print(env.get_legal_actions(state))

[False False False False False False False False False False False False
 False  True False False False  True False False]


In [22]:
        obs=np.full((2*5,),True)

        print(obs)
        # set skip timestep to False
        obs[5:]=False
        print(obs)

[ True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True False False False False False]


In [23]:
for i in range(5,10):
    print(i)

5
6
7
8
9


In [1]:
print(env.get_legal_actions(state))
print(env.current_machines_status)
print(env.timestemp_list)
print(env.current_timestep)
print(env.action_space.n)
env.render()

NameError: name 'env' is not defined

In [4]:
env.render(start_count=1)
print(env.get_legal_actions(state))

[ True  True  True  True  True  True  True  True False  True False]


In [4]:
df,px=env.render_to_df(start_count=1)

In [6]:
import plotly.express as px
x_bar="Machine"
y_bar="Job"
df.sort_values(inplace=True,by="Job")
fig = px.timeline(df, x_start="Start", x_end="Finish", color=x_bar, y=y_bar)

fig.show()

In [7]:
d = {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 34), 'Machine': 'machine_0'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 59), 'Finish': datetime.datetime(2022, 1, 1, 0, 2, 58), 'Machine': 'machine_0'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 2, 58), 'Finish': datetime.datetime(2022, 1, 1, 0, 4, 21), 'Machine': 'machine_0'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 8, 52), 'Finish': datetime.datetime(2022, 1, 1, 0, 10, 31), 'Machine': 'machine_0'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 10, 31), 'Finish': datetime.datetime(2022, 1, 1, 0, 12, 6), 'Machine': 'machine_0'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 12, 6), 'Finish': datetime.datetime(2022, 1, 1, 0, 13, 8), 'Machine': 'machine_0'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 13, 8), 'Finish': datetime.datetime(2022, 1, 1, 0, 14, 24), 'Machine': 'machine_0'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 14, 45), 'Finish': datetime.datetime(2022, 1, 1, 0, 16, 11), 'Machine': 'machine_0'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 19, 4), 'Finish': datetime.datetime(2022, 1, 1, 0, 20, 36), 'Machine': 'machine_0'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 22, 57), 'Finish': datetime.datetime(2022, 1, 1, 0, 24, 25), 'Machine': 'machine_0'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 39), 'Machine': 'machine_1'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 39), 'Finish': datetime.datetime(2022, 1, 1, 0, 3, 5), 'Machine': 'machine_1'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 3, 5), 'Finish': datetime.datetime(2022, 1, 1, 0, 4, 27), 'Machine': 'machine_1'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 4, 27), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 28), 'Machine': 'machine_1'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 5, 28), 'Finish': datetime.datetime(2022, 1, 1, 0, 6, 33), 'Machine': 'machine_1'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 6, 33), 'Finish': datetime.datetime(2022, 1, 1, 0, 7, 44), 'Machine': 'machine_1'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 7, 44), 'Finish': datetime.datetime(2022, 1, 1, 0, 8, 50), 'Machine': 'machine_1'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 8, 50), 'Finish': datetime.datetime(2022, 1, 1, 0, 9, 57), 'Machine': 'machine_1'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 16, 46), 'Finish': datetime.datetime(2022, 1, 1, 0, 18, 12), 'Machine': 'machine_1'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 20, 36), 'Finish': datetime.datetime(2022, 1, 1, 0, 21, 58), 'Machine': 'machine_1'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 2, 24), 'Finish': datetime.datetime(2022, 1, 1, 0, 3, 32), 'Machine': 'machine_2'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 5, 32), 'Finish': datetime.datetime(2022, 1, 1, 0, 6, 23), 'Machine': 'machine_2'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 8, 50), 'Finish': datetime.datetime(2022, 1, 1, 0, 10, 20), 'Machine': 'machine_2'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 10, 20), 'Finish': datetime.datetime(2022, 1, 1, 0, 11, 49), 'Machine': 'machine_2'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 11, 49), 'Finish': datetime.datetime(2022, 1, 1, 0, 12, 56), 'Machine': 'machine_2'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 12, 56), 'Finish': datetime.datetime(2022, 1, 1, 0, 14, 30), 'Machine': 'machine_2'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 14, 30), 'Finish': datetime.datetime(2022, 1, 1, 0, 15, 25), 'Machine': 'machine_2'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 15, 25), 'Finish': datetime.datetime(2022, 1, 1, 0, 17, 1), 'Machine': 'machine_2'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 17, 1), 'Finish': datetime.datetime(2022, 1, 1, 0, 18, 7), 'Machine': 'machine_2'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 18, 7), 'Finish': datetime.datetime(2022, 1, 1, 0, 19, 27), 'Machine': 'machine_2'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 9), 'Machine': 'machine_3'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 9), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 59), 'Machine': 'machine_3'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 59), 'Finish': datetime.datetime(2022, 1, 1, 0, 2, 49), 'Machine': 'machine_3'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 2, 49), 'Finish': datetime.datetime(2022, 1, 1, 0, 4, 10), 'Machine': 'machine_3'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 4, 10), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 32), 'Machine': 'machine_3'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 7, 32), 'Finish': datetime.datetime(2022, 1, 1, 0, 9, 8), 'Machine': 'machine_3'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 9, 11), 'Finish': datetime.datetime(2022, 1, 1, 0, 10, 5), 'Machine': 'machine_3'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 15, 25), 'Finish': datetime.datetime(2022, 1, 1, 0, 16, 42), 'Machine': 'machine_3'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 16, 42), 'Finish': datetime.datetime(2022, 1, 1, 0, 18, 14), 'Machine': 'machine_3'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 21, 38), 'Finish': datetime.datetime(2022, 1, 1, 0, 22, 57), 'Machine': 'machine_3'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 38), 'Machine': 'machine_4'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 38), 'Finish': datetime.datetime(2022, 1, 1, 0, 3, 6), 'Machine': 'machine_4'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 3, 6), 'Finish': datetime.datetime(2022, 1, 1, 0, 4, 27), 'Machine': 'machine_4'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 4, 27), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 55), 'Machine': 'machine_4'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 5, 55), 'Finish': datetime.datetime(2022, 1, 1, 0, 7, 32), 'Machine': 'machine_4'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 7, 32), 'Finish': datetime.datetime(2022, 1, 1, 0, 9, 11), 'Machine': 'machine_4'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 9, 11), 'Finish': datetime.datetime(2022, 1, 1, 0, 10, 26), 'Machine': 'machine_4'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 10, 26), 'Finish': datetime.datetime(2022, 1, 1, 0, 11, 47), 'Machine': 'machine_4'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 11, 47), 'Finish': datetime.datetime(2022, 1, 1, 0, 13, 3), 'Machine': 'machine_4'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 13, 3), 'Finish': datetime.datetime(2022, 1, 1, 0, 14, 28), 'Machine': 'machine_4'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 12), 'Machine': 'machine_5'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 4, 27), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 31), 'Machine': 'machine_5'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 6, 41), 'Finish': datetime.datetime(2022, 1, 1, 0, 8, 20), 'Machine': 'machine_5'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 8, 20), 'Finish': datetime.datetime(2022, 1, 1, 0, 9, 54), 'Machine': 'machine_5'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 9, 54), 'Finish': datetime.datetime(2022, 1, 1, 0, 11, 19), 'Machine': 'machine_5'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 11, 20), 'Finish': datetime.datetime(2022, 1, 1, 0, 12, 26), 'Machine': 'machine_5'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 12, 26), 'Finish': datetime.datetime(2022, 1, 1, 0, 13, 25), 'Machine': 'machine_5'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 13, 25), 'Finish': datetime.datetime(2022, 1, 1, 0, 14, 23), 'Machine': 'machine_5'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 14, 31), 'Finish': datetime.datetime(2022, 1, 1, 0, 15, 39), 'Machine': 'machine_5'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 18, 7), 'Finish': datetime.datetime(2022, 1, 1, 0, 19, 46), 'Machine': 'machine_5'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 34), 'Finish': datetime.datetime(2022, 1, 1, 0, 2, 45), 'Machine': 'machine_6'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 2, 45), 'Finish': datetime.datetime(2022, 1, 1, 0, 3, 58), 'Machine': 'machine_6'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 3, 58), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 7), 'Machine': 'machine_6'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 5, 7), 'Finish': datetime.datetime(2022, 1, 1, 0, 6, 41), 'Machine': 'machine_6'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 6, 41), 'Finish': datetime.datetime(2022, 1, 1, 0, 7, 45), 'Machine': 'machine_6'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 7, 45), 'Finish': datetime.datetime(2022, 1, 1, 0, 8, 51), 'Machine': 'machine_6'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 8, 53), 'Finish': datetime.datetime(2022, 1, 1, 0, 9, 51), 'Machine': 'machine_6'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 10, 5), 'Finish': datetime.datetime(2022, 1, 1, 0, 11, 20), 'Machine': 'machine_6'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 12, 56), 'Finish': datetime.datetime(2022, 1, 1, 0, 14, 31), 'Machine': 'machine_6'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 19, 46), 'Finish': datetime.datetime(2022, 1, 1, 0, 20, 38), 'Machine': 'machine_6'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 0, 50), 'Machine': 'machine_7'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 0, 50), 'Finish': datetime.datetime(2022, 1, 1, 0, 2, 24), 'Machine': 'machine_7'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 4, 10), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 35), 'Machine': 'machine_7'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 5, 35), 'Finish': datetime.datetime(2022, 1, 1, 0, 6, 31), 'Machine': 'machine_7'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 9, 54), 'Finish': datetime.datetime(2022, 1, 1, 0, 11, 19), 'Machine': 'machine_7'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 11, 19), 'Finish': datetime.datetime(2022, 1, 1, 0, 12, 37), 'Machine': 'machine_7'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 13, 6), 'Finish': datetime.datetime(2022, 1, 1, 0, 14, 45), 'Machine': 'machine_7'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 15, 39), 'Finish': datetime.datetime(2022, 1, 1, 0, 16, 46), 'Machine': 'machine_7'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 19, 27), 'Finish': datetime.datetime(2022, 1, 1, 0, 20, 36), 'Machine': 'machine_7'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 21, 58), 'Finish': datetime.datetime(2022, 1, 1, 0, 23, 32), 'Machine': 'machine_7'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 1, 23), 'Finish': datetime.datetime(2022, 1, 1, 0, 2, 24), 'Machine': 'machine_8'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 3, 6), 'Finish': datetime.datetime(2022, 1, 1, 0, 4, 14), 'Machine': 'machine_8'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 4, 27), 'Finish': datetime.datetime(2022, 1, 1, 0, 5, 34), 'Machine': 'machine_8'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 7, 17), 'Finish': datetime.datetime(2022, 1, 1, 0, 8, 52), 'Machine': 'machine_8'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 8, 52), 'Finish': datetime.datetime(2022, 1, 1, 0, 10, 12), 'Machine': 'machine_8'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 12, 6), 'Finish': datetime.datetime(2022, 1, 1, 0, 13, 43), 'Machine': 'machine_8'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 13, 43), 'Finish': datetime.datetime(2022, 1, 1, 0, 15, 18), 'Machine': 'machine_8'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 15, 18), 'Finish': datetime.datetime(2022, 1, 1, 0, 16, 51), 'Machine': 'machine_8'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 16, 51), 'Finish': datetime.datetime(2022, 1, 1, 0, 17, 58), 'Machine': 'machine_8'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 17, 58), 'Finish': datetime.datetime(2022, 1, 1, 0, 19, 4), 'Machine': 'machine_8'}, {'Job': 'job_2', 'Start': datetime.datetime(2022, 1, 1, 0, 0), 'Finish': datetime.datetime(2022, 1, 1, 0, 1, 23), 'Machine': 'machine_9'}, {'Job': 'job_4', 'Start': datetime.datetime(2022, 1, 1, 0, 5, 55), 'Finish': datetime.datetime(2022, 1, 1, 0, 7, 17), 'Machine': 'machine_9'}, {'Job': 'job_9', 'Start': datetime.datetime(2022, 1, 1, 0, 7, 17), 'Finish': datetime.datetime(2022, 1, 1, 0, 8, 53), 'Machine': 'machine_9'}, {'Job': 'job_0', 'Start': datetime.datetime(2022, 1, 1, 0, 11, 49), 'Finish': datetime.datetime(2022, 1, 1, 0, 13, 6), 'Machine': 'machine_9'}, {'Job': 'job_3', 'Start': datetime.datetime(2022, 1, 1, 0, 14, 24), 'Finish': datetime.datetime(2022, 1, 1, 0, 15, 27), 'Machine': 'machine_9'}, {'Job': 'job_7', 'Start': datetime.datetime(2022, 1, 1, 0, 15, 27), 'Finish': datetime.datetime(2022, 1, 1, 0, 16, 46), 'Machine': 'machine_9'}, {'Job': 'job_8', 'Start': datetime.datetime(2022, 1, 1, 0, 16, 51), 'Finish': datetime.datetime(2022, 1, 1, 0, 18, 28), 'Machine': 'machine_9'}, {'Job': 'job_5', 'Start': datetime.datetime(2022, 1, 1, 0, 20, 36), 'Finish': datetime.datetime(2022, 1, 1, 0, 21, 38), 'Machine': 'machine_9'}, {'Job': 'job_6', 'Start': datetime.datetime(2022, 1, 1, 0, 21, 38), 'Finish': datetime.datetime(2022, 1, 1, 0, 22, 49), 'Machine': 'machine_9'}, {'Job': 'job_1', 'Start': datetime.datetime(2022, 1, 1, 0, 23, 32), 'Finish': datetime.datetime(2022, 1, 1, 0, 24, 35), 'Machine': 'machine_9'}
sd = sorted(d.items())

for k,v in sd:
    print (k, v)

NameError: name 'datetime' is not defined

In [3]:
env.render(y_bar="Job",x_bar="Machine")

In [6]:
env.render()

In [18]:
for i in range(prod.shape[0]):
    for j in range(prod.shape[1]-1):
        if prod[i,j][3]<=prod[i,j+1][2]:
            print(prod[i,j][3])
        else:
            print(i,j)

19
48
70
111
126
212
320
358
378
402
441
467
486
524
563
582
598
655
679
18
69
104
125
225
279
309
339
375
423
462
484
515
542
572
635
683
713
856
12
59
91
148
245
273
364
402
417
446
475
501
528
568
597
632
650
681
712
25
62
78
114
136
182
201
241
280
291
303
331
388
405
425
465
579
638
704
145
200
232
244
285
298
320
341
391
432
444
456
468
485
545
566
599
680
715
120
143
165
192
232
245
277
316
328
354
382
396
426
443
472
512
523
557
605
39
71
104
123
141
183
210
235
252
277
299
330
355
390
426
459
472
630
746
55
117
171
213
244
283
306
324
399
427
481
557
581
610
643
675
688
726
776
31
48
86
106
117
155
176
190
222
247
262
278
312
332
354
373
386
538
673
11
52
65
127
175
186
213
243
281
293
327
346
395
418
456
474
542
586
602
23
45
121
135
152
175
207
246
280
355
379
408
438
471
502
538
563
597
633
18
37
48
59
83
245
305
330
352
374
406
446
488
526
554
591
652
687
742
32
70
92
117
153
170
187
211
222
358
438
479
519
536
596
628
647
668
739
32
83
119
159
177
191
212
229
261
302
323


In [2]:
prod=(env.production_list)   

In [4]:
print(action)

print(env.get_legal_actions(state))
print(env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]])
cur_machine=env.current_machines_status
job_ma=env.job_machine_matrix
print(env.count_finished_tasks_job_matrix[action])
#print(env.current_machines_status[env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]],:])
#print(env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]])
#print(env.current_machines_status)
env.render()


10
[False False False False False False False False False False False]


IndexError: index 10 is out of bounds for axis 0 with size 10

In [2]:
print(env.n_machines)
print(env.n_jobs)
print(env.n_tasks)
print(env.count_finished_tasks_machine_matrix)
print(env.production_list)    
env.render()


15
20
15
[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
[[(0, 0, 0, 19) None None None None None None None None None None None
  None None None]
 [(9, 0, 0, 18) None None None None None None None None None None None
  None None None]
 [(19, 0, 0, 12) None None None None None None None None None None None
  None None None]
 [(2, 0, 0, 25) None None None None None None None None None None None
  None None None]
 [None None None None None None None None None None None None None None
  None]
 [None None None None None None None None None None None None None None
  None]
 [(11, 0, 0, 32) None None None None None None None None None None None
  None None None]
 [None None None None None None None None None None None None None None
  None]
 [(15, 0, 0, 38) None None None None None None None None None None None
  None None None]
 [(1, 0, 0, 11) None None None None None None None None None None None
  None None None]
 [(5, 0, 0, 23) None None None None None None None None None None None
  None 

In [ ]:
print(sum(env.count_finished_tasks_job_matrix))
print(sum(env.count_finished_tasks_machine_matrix))

In [ ]:
env.render()
print(env.production_list)

In [ ]:
import ale_py as ale
ale.importroms('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/roms/Space Invaders.bin')
import gym
env = gym.make('SpaceInvaders-v4')
env.reset()
env.render()


In [35]:
import JSSEnv
import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
import src.jsp_instance_parser 
import time
import gym
import pandas as pd
import random
import numpy as np
from src.jss_lite.jss_lite import jss_lite

index=['klagenfurt','alex','lite']
df=pd.DataFrame(np.zeros((3,5)),columns=['setuptime','resettime','50xactiontime','col4','col5'],index=index)
#df=pd.DataFrame
#df.set_index('index')
for i in range(100):

    def timer(innerfun):
        tmp_t=time.time()
        tmp_r=innerfun()
        return time.time()-tmp_t,tmp_r

    tmp_t=time.time()
    env2=gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/abz8.txt'})
    t_1=(time.time()-tmp_t)
    df['setuptime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    env_3=jss_lite()
    t_1=time.time()-tmp_t
    df['setuptime']['lite']+=t_1/100




    tmp_t=time.time()
    path='resources/jsp_instances/standard/abz8.txt'
    curr_instance=src.jsp_instance_parser.parse_jps_standard_specification(path)
    res,std_matrix=curr_instance
    env_alex = jss_env.DisjunctiveGraphJssEnv(res,default_visualisations='gantt_console')
    t_2=(time.time()-tmp_t)
    df['setuptime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env2.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    t2=env_alex.reset()
    t_2=(time.time()-tmp_t)
    df['resettime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env_3.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['lite']+=t_1/100


    state=env2.reset()
    for i in range(50):
        legal_action=state['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env2.step(action)
        t_1=(time.time()-tmp_t)
        df['50xactiontime']['klagenfurt']+=t_1/50

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break
    state=env_3.reset()
    for i in range(50):
        legal_action=env_3.get_legal_actions(state)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env_3.step(action)
        t_1=(time.time()-tmp_t)
        df['50xactiontime']['lite']+=t_1/50

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break



    action_list=np.arange(0,env_alex.n_jobs*env_alex.n_machines)
    for i in range(50):
        action_mask=env_alex.valid_action_mask()
        actions=action_list[action_mask]
        next_action=random.choice(actions)
        #next_action=actions[action_mask[0]]
        #print(next_action)
        tmp_t=time.time()
        state,reward,done,info =env_alex.step(next_action)
        t_2=(time.time()-tmp_t)
        df['50xactiontime']['alex']+=t_2/50
        if done == True:
            print(f"finished after {i} steps")
            break
        #env.render()






[18:23:14] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

           INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

           INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

...

           INFO     instance appears to contain 20 jobs, 15 machines

[18:23:47] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

           INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

In [ ]:
import resource
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
dt = datetime(2022, 1, 1, 0, 0, 0)
#stores quatuple(job,task,start_time,finish_time) 
time_now=time.time()
production_list=env_3.production_list
def production_to_dict(input,i,j):
    return(dict(Task=f"job_{input[0]}", Start=dt + timedelta(seconds=int(input[2])),Finish=dt + timedelta(seconds=int(input[3])),Resource=f"machine_{i}"))
df=pd.DataFrame()
liste=[]
for i in range(10):
    for j in range(10):
        if production_list[i,j] is not None:
            liste.append(production_to_dict(production_list[i,j],i,j))

df_2=pd.DataFrame(liste)
fig = px.timeline(df_2, x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.show()

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])

#fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
#fig.show()